In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import urljoin
import numpy as np

### First Attempt just using Beautiful Soup

In [ ]:
levels = ['college', 'high-school']
sports = ['football', 'basketball', 'baseball', 'womens-basketball', 'volleyball', 'gymnastics',
         'mens-lacrosse', 'womens-lacrosse', 'mens-soccer', 'womens-soccer', 'softball', 'womens-track',
         'mens-golf', 'womens-golf', 'mens-hockey', 'womens-hockey', 'mens-swimming', 'womens-swimming']

all_names = []

for level in levels:
    for sport in sports:

        URL = f'https://www.on3.com/nil/rankings/player/{level}/{sport}/'

        page = requests.get(URL)
        soup = BeautifulSoup(page.content, 'html.parser')
        results = soup.find(class_="NilRankingsPageComponent_nilRankingsList__t14Ao")

        try:
            names_sport = results.find_all('a', class_="MuiTypography-root MuiLink-root MuiLink-underlineNone MuiTypography-h5 MuiTypography-colorPrimary")
            for name in names_sport:
                all_names.append(name)
        except:
            # print("Whoops! No athletes appear to have NIL deal for {} {}".format(level, sport))
            continue
    #end
#end

In [7]:
print(len(all_names))

414


It looks like there were only 414 athletes using this section of the On3 database... This is such a small number of athletes that we should investiage looking at different sections of the website

### Using Selenium & Beatuiful Soup

In [34]:
sports = ['football']
#years = ['2022','2023','2024','2025','2026']
years = ['2023']

exp = []
athlete_grade = []
ages = []
ranks = []
high_school = []
home_town = []
colleges = []


for year in years:
    for sport in sports:
        
        # Selenium Driver to click on "Load More"
        driver = webdriver.Chrome()
        URL = f'https://www.on3.com/db/rankings/industry-player/{sport}/{year}/'
        driver.get(URL)
        
        # while True:
        #     try:
        #         load_more_button = WebDriverWait(driver, 10).until(
        #             EC.element_to_be_clickable((By.XPATH, "//span[@class='MuiButton-label' and contains(text(), 'Load More')]"))
        #         )
        #         load_more_button.click()
        #         time.sleep(10)
        #     except:
        #         break
        
        # Beautiful Soup - for each player
        page_source = driver.page_source
        page_soup = BeautifulSoup(page_source, 'html.parser')
        results = page_soup.find(class_="PlayerRankings_playerRankings__jvfFg")

        athletes = results.find_all('a', class_="MuiTypography-root MuiLink-root MuiLink-underlineHover MuiTypography-h5 MuiTypography-colorPrimary")
        # It is possible that two athletes have the same name: John Smith
        # We cannot use a dictionary so a list of tuples is used for now
        names = [athlete.text for athlete in athletes]

        # Generate a list of links that we can iterate through
        links = [athlete['href'] for athlete in athletes]
        base_url = "https://www.on3.com/"
        athlete_links = [urljoin(base_url, link) for link in links]

        for athlete_link in athlete_links:
            # We can now go into each individual athletes page
            # for that sport, for that year, after loading more
            try:
                driver.get(athlete_link)
                time.sleep(3)

                athlete_source = driver.page_source
                page_soup = BeautifulSoup(athlete_source, 'html.parser')

                CollegeRankingInfo = page_soup.find(class_='CollegeRanking_info__LM3nn')

                if CollegeRankingInfo:
                    exp_year = CollegeRankingInfo.find_all(class_='MuiTypography-root CollegeRanking_span__qtAfW MuiTypography-subtitle1 MuiTypography-colorTextPrimary')
                    if len(exp_year) == 2:
                        exp.append(exp_year[0].text)
                        athlete_grade.append(exp_year[1].text)
                    else:
                        exp.append(np.nan)
                        athlete_grade.append(np.nan)
                    age = CollegeRankingInfo.find(class_='MuiTypography-root CollegeRanking_span__qtAfW MuiTypography-subtitle1 MuiTypography-colorPrimary')
                    ages.append(age.text)
                else:
                    exp.append(np.nan)
                    athlete_grade.append(np.nan)
                    ages.append(np.nan)
                
                RecruitModuleInfo = page_soup.find(class_='RecruitModule_info__Ugxqd')

                if RecruitModuleInfo:
                    ranking = RecruitModuleInfo.find(class_='RecruitModule_rating__sONqb')
                    ranks.append(ranking.text)

                    homeInfo = RecruitModuleInfo.find_all(class_='MuiTypography-root RecruitModule_span__KmmzN MuiTypography-subtitle1 MuiTypography-colorTextPrimary')
                    high_school.append(homeInfo[-2].text)
                    home_town.append(homeInfo[-1].text)
                else:
                    ranks.append(np.nan)
                    high_school.append(np.nan)
                    home_town.append(np.nan)
            except:
                print("Link {} is no good - removing athlete altogether".format(athlete_link))
                badLink = athlete_link
                index = athlete_link.index(badLink)
                names.pop(index)
                continue
            
            try:
                # Move to Recruiting Page
                driver.get(urljoin(athlete_link,'recruiting/'))
                time.sleep(3)
                athlete_source = driver.page_source
                page_soup = BeautifulSoup(athlete_source, 'html.parser')
                
                # Find url for all teams and navigate to page
                all_team_link = page_soup.find(class_='MuiTypography-root MuiLink-root MuiLink-underlineHover PlayerInterestsModule_text__kjqNU MuiTypography-caption MuiTypography-colorPrimary')
                driver.get(urljoin(base_url,all_team_link['href']))
                time.sleep(3)
                athlete_source = driver.page_source
                page_soup = BeautifulSoup(athlete_source, 'html.parser')
                
                #RecruitTeamsInfo = page_soup.find(class_='DatabaseLayout_mainmiddle__b010n')
                RecruitColleges = page_soup.find_all(class_='PlayerInterestsItem_teamContainer__vjQkf')
                dummy_colleges=[]
                for college in RecruitColleges:
                    college_name = college.find(class_='MuiTypography-root MuiLink-root MuiLink-underlineNone PlayerInterestsItem_teamName__FeBHv MuiTypography-h5 MuiTypography-colorPrimary')
                    college_status = college.find(class_='MuiTypography-root PlayerInterestsItem_status__1_rA8 MuiTypography-subtitle1 MuiTypography-colorTextPrimary')
                    college_dist = college.find(class_='MuiTypography-root PlayerInterestsItem_distanceText__KJhj3 MuiTypography-caption MuiTypography-colorTextPrimary')
                    dummy_colleges.append([college_name,college_status,college_dist])
                colleges.append(dummy_colleges)
                print(len(colleges[0]))
                
                # college_names = RecruitTeamsInfo.find_all(class_='MuiTypography-root MuiLink-root MuiLink-underlineNone PlayerInterestsItem_teamName__FeBHv MuiTypography-h5 MuiTypography-colorPrimary')
                # college_status = RecruitTeamsInfo.find_all(class_='MuiTypography-root PlayerInterestsItem_status__1_rA8 MuiTypography-subtitle1 MuiTypography-colorTextPrimary')
                # print(college_status)
                # player_colleges = []
                # player_college_status = []
                # for college in college_names:
                #     player_colleges.append(college.text)
                # for college in college_status:
                #     player_college_status.append(college.text)
                # colleges.append(player_colleges)
                # colleges_statuses.append(player_college_status)
                    
                # print(colleges_statuses)
                #print(college_names.text)
            except:
                print("Error on recruiting page")
        #end
        print(len(names), len(exp), len(athlete_grade), len(ages), len(ranks), len(high_school), len(home_town))
        print(names[0], exp[0], athlete_grade[0], ages[0], ranks[0], high_school[0], home_town[0])
    #end
#end

        # This successfully takes us back to the previous page, but it might require us to reload every athlete...
        # driver.back()
        # url = driver.current_url
        # print(url)

driver.quit()

[<li class="PlayerInterestsItem_teamContainer__vjQkf"><div class="PlayerInterestsItem_teamWrapper__VfM0b"><a class="MuiTypography-root MuiLink-root MuiLink-underlineHover PlayerInterestsItem_teamLogoContainer__EONe5 MuiTypography-colorPrimary" href="/college/texas-longhorns/football/2023/industry-comparison-commits/"><img alt="Team Avatar" class="PlayerInterestsItem_teamLogo__Q24kB" height="50" loading="lazy" src="https://on3static.com/uploads/assets/276/150/150276.svg" title="Team" width="50"/></a><div class="PlayerInterestsItem_teamNameContainer__e_AOb"><a class="MuiTypography-root MuiLink-root MuiLink-underlineNone PlayerInterestsItem_teamName__FeBHv MuiTypography-h5 MuiTypography-colorPrimary" href="/college/texas-longhorns/football/2023/industry-comparison-commits/">Texas</a><h6 class="MuiTypography-root PlayerInterestsItem_status__1_rA8 MuiTypography-subtitle1 MuiTypography-colorTextPrimary">Enrolled</h6></div></div><div class="PlayerInterestsItem_predictionContainer__R8PBA"><h6 

In [16]:
print(len(names), len(exp), len(athlete_grade), len(ages), len(ranks), len(high_school), len(home_town))
print(names[253], exp[-2], athlete_grade[-2], ages[-2], ranks[-2], high_school[-2], home_town[-2])

326 254 254 254 254 254 254
Malik Olafioye 2023 - present Freshman - 85.30 Ravenscroft School Raleigh, NC


In [17]:
for athlete_link in athlete_links[250:260]:
    print(athlete_link)

https://www.on3.com/db/bobby-durkin-156822/
https://www.on3.com/db/john-blackwell-153825/
https://www.on3.com/db/comeh-emuobor-147246/
https://www.on3.com/db/malik-olafioye-81952/
https://www.on3.com/db/wesley-tubbs-iii-150899/
https://www.on3.com/db/jalen-hooks-54341/
https://www.on3.com/db/austin-ball-147266/
https://www.on3.com/db/chuck-bailey-iii-147243/
https://www.on3.com/db/gabe-sisk-44602/
https://www.on3.com/db/isaiah-manning-147244/


In [12]:
# sports = ['basketball']
# #years = ['2022','2023','2024','2025','2026']
# years = ['2023']


# urls = {}

# for year in years:
#     for sport in sports:

        
            
        # page = requests.get(URL)
        # soup = BeautifulSoup(page.content, 'html.parser')
        #results = soup.find(class_="PlayerRankings_playerRankings__jvfFg")
        
        # page_soup = soup(driver.page_source, 'html.parser')
        # print(page_soup)
        # results = page_soup.find(class_="PlayerRankings_playerRankings__jvfFg")
        # try:
        #     names_sport = results.find_all('a', class_="MuiTypography-root MuiLink-root MuiLink-underlineHover MuiTypography-h5 MuiTypography-colorPrimary")
        #     for name in names_sport:
        #         urls[name.text] = name['href']
        # except:
        #     print("Whoops! No athletes appear to have NIL deal for {} {}".format(level, sport))
        #     continue
    #end
#end

In [13]:
# all_info = {}

# for name, url in urls.items():
#     all_info[name] = []
#     URL = f'https://www.on3.com{url}/recruiting'
#     page = requests.get(URL)
#     soup = BeautifulSoup(page.content, 'html.parser')
    
    
    
#     results = soup.find(class_="PlayerRecruiting_playergrid__wYlb_")
#     player_info = results.find_all('span', class_="MuiTypography-root MeasurementInfo_text__dCryI MuiTypography-body1 MuiTypography-colorTextPrimary")
#     for info in player_info:
#         all_info[name].append(info.text)
    
#     results = soup.find(class_="Rankings_container__U2afk")
#     rankings_info = results.find_all('p', class_="MuiTypography-root Rankings_industryRating__9uavm MuiTypography-body1 MuiTypography-colorTextPrimary")
#     for info in rankings_info:
#         all_info[name].append(info.text)
    
    
#     results = soup.find(class_="PlayerInterestsModule_targets__rxz4U")
#     recruitment_info = results.find_all(['a','h6'], class_=["MuiTypography-root MuiLink-root MuiLink-underlineNone PlayerInterestsItem_teamName__FeBHv MuiTypography-h5 MuiTypography-colorPrimary","MuiTypography-root PlayerInterestsItem_predictionLabel__mT201 MuiTypography-subtitle1 MuiTypography-colorTextPrimary"])
#     for info in  recruitment_info:
#         all_info[name].append(info.text)
